In [103]:
import pandas as pd
import numpy as np
import sys
import re 
import datetime
%matplotlib inline
from matplotlib import pyplot as plt
import sklearn
from sklearn.decomposition import PCA
from sklearn.mixture import GMM
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
from sklearn.metrics import classification_report
print(sys.version)

3.5.2 |Anaconda 4.1.1 (x86_64)| (default, Jul  2 2016, 17:52:12) 
[GCC 4.2.1 Compatible Apple LLVM 4.2 (clang-425.0.28)]


In [104]:
def day_time(x):
    y = ''
    if datetime.datetime.fromtimestamp(x['unix_timestamp_of_request_utc']).hour < 10:
        y = 'morning'
    elif datetime.datetime.fromtimestamp(x['unix_timestamp_of_request_utc']).hour >= 10 and datetime.datetime.fromtimestamp(x['unix_timestamp_of_request_utc']).hour < 16:
        y = 'midday'
    elif datetime.datetime.fromtimestamp(x['unix_timestamp_of_request_utc']).hour >= 16 and datetime.datetime.fromtimestamp(x['unix_timestamp_of_request_utc']).hour < 21:
        y = 'evening'
    else: 
        y = 'late_night'
    return y

day_values = ['morning', 'midday', 'evening', 'late_night']
    
def human_time(a):
    import datetime
    from datetime import date
    import calendar
    ### for the data in raop, return human time.  maybe the time of day matters
    a['human_readable_local_time'] = datetime.datetime.fromtimestamp(a['unix_timestamp_of_request']).strftime('%Y-%m-%d %H:%M:%S')
    a['human_readable_UTC_time'] = datetime.datetime.fromtimestamp(a['unix_timestamp_of_request_utc']).strftime('%Y-%m-%d %H:%M:%S')
    a['weekday'] = calendar.day_name[datetime.datetime.fromtimestamp(a['unix_timestamp_of_request']).weekday()].lower()
    a['month'] = datetime.datetime.fromtimestamp(a['unix_timestamp_of_request']).strftime("%B").lower()
    a['time_of_day'] = day_time(a)
    return a

In [105]:
a = pd.read_json('train.json')

In [106]:
b = a.apply(human_time, axis=1)

In [107]:
b.head()

,giver_username_if_known,number_of_downvotes_of_request_at_retrieval,number_of_upvotes_of_request_at_retrieval,post_was_edited,request_id,request_number_of_comments_at_retrieval,request_text,request_text_edit_aware,request_title,requester_account_age_in_days_at_request,...,requester_upvotes_plus_downvotes_at_retrieval,requester_user_flair,requester_username,unix_timestamp_of_request,unix_timestamp_of_request_utc,human_readable_local_time,human_readable_UTC_time,weekday,month,time_of_day
0,N/A,0,1,0,t3_l25d7,0,Hi I am in need of food for my 4 children we a...,Hi I am in need of food for my 4 children we a...,Request Colorado Springs Help Us Please,0.000000,...,1,None,nickylvst,1317852607,1317849007,2011-10-05 15:10:07,2011-10-05 14:10:07,wednesday,october,midday
1,N/A,2,5,0,t3_rcb83,0,I spent the last money I had on gas today. Im ...,I spent the last money I had on gas today. Im ...,"[Request] California, No cash and I could use ...",501.111100,...,11168,None,fohacidal,1332652424,1332648824,2012-03-24 22:13:44,2012-03-24 21:13:44,saturday,march,late_night
2,N/A,0,3,0,t3_lpu5j,0,My girlfriend decided it would be a good idea ...,My girlfriend decided it would be a good idea ...,"[Request] Hungry couple in Dundee, Scotland wo...",0.000000,...,3,None,jacquibatman7,1319650094,1319646494,2011-10-26 10:28:14,2011-10-26 09:28:14,wednesday,october,morning
3,N/A,0,1,1,t3_mxvj3,4,"It's cold, I'n hungry, and to be completely ho...","It's cold, I'n hungry, and to be completely ho...","[Request] In Canada (Ontario), just got home f...",6.518438,...,81,None,4on_the_floor,1322855434,1322855434,2011-12-02 11:50:34,2011-12-02 11:50:34,friday,december,midday
4,N/A,6,6,0,t3_1i6486,5,hey guys:\n I love this sub. I think it's grea...,hey guys:\n I love this sub. I think it's grea...,[Request] Old friend coming to visit. Would LO...,162.063252,...,1887,None,Futuredogwalker,1373657691,1373654091,2013-07-12 12:34:51,2013-07-12 11:34:51,friday,july,midday


In [108]:
c = pd.read_json('test.json')

In [109]:
c.head()

,giver_username_if_known,request_id,request_text_edit_aware,request_title,requester_account_age_in_days_at_request,requester_days_since_first_post_on_raop_at_request,requester_number_of_comments_at_request,requester_number_of_comments_in_raop_at_request,requester_number_of_posts_at_request,requester_number_of_posts_on_raop_at_request,requester_number_of_subreddits_at_request,requester_subreddits_at_request,requester_upvotes_minus_downvotes_at_request,requester_upvotes_plus_downvotes_at_request,requester_username,unix_timestamp_of_request,unix_timestamp_of_request_utc
0,N/A,t3_i8iy4,Hey all! It's about 95 degrees here and our ki...,[request] pregger gf 95 degree house and no fo...,42.083866,0.0,57,0,10,0,16,"[AskReddit, COents, Denver, DenverBroncos, Lib...",364,840,j_like,1308963419,1308959819
1,N/A,t3_1mfqi0,I didn't know a place like this exists! \n\nI ...,"[Request] Lost my job day after labour day, st...",223.784537,0.0,145,0,36,0,29,"[Android, AskReddit, GrandTheftAutoV, IAmA, Mi...",516,1448,0110110101101100,1379263523,1379259923
2,N/A,t3_lclka,Hi Reddit. Im a single dad having a really rou...,(Request) pizza for my kids please?,0.000000,0.0,0,0,0,0,0,[],0,0,singledad22601,1318636421,1318632821
3,N/A,t3_1jdgdj,Hi I just moved to Waltham MA from my home sta...,[Request] Just moved to a new state(Waltham MA...,481.311273,0.0,277,0,17,0,30,"[AdviceAnimals, Art, AskReddit, GetMotivated, ...",1058,2062,Neuronut,1375220282,1375216682
4,N/A,t3_t2qt4,We're just sitting here near indianapolis on o...,"[Request] Two girls in between paychecks, we'v...",0.000000,0.0,0,0,0,0,0,[],0,0,so_damn_hungry,1335934358,1335930758


In [110]:
#get the columns that are actually in the test set
df = b[list(c.columns)]

In [111]:
tstdf = c[list(c.columns)]

In [112]:
df1 = df.apply(human_time, axis = 1)
tstdf1 = tstdf.apply(human_time, axis = 1)

In [113]:
df1

,giver_username_if_known,request_id,request_text_edit_aware,request_title,requester_account_age_in_days_at_request,requester_days_since_first_post_on_raop_at_request,requester_number_of_comments_at_request,requester_number_of_comments_in_raop_at_request,requester_number_of_posts_at_request,requester_number_of_posts_on_raop_at_request,...,requester_upvotes_minus_downvotes_at_request,requester_upvotes_plus_downvotes_at_request,requester_username,unix_timestamp_of_request,unix_timestamp_of_request_utc,human_readable_local_time,human_readable_UTC_time,weekday,month,time_of_day
0,N/A,t3_l25d7,Hi I am in need of food for my 4 children we a...,Request Colorado Springs Help Us Please,0.000000,0.000000,0,0,0,0,...,0,0,nickylvst,1317852607,1317849007,2011-10-05 15:10:07,2011-10-05 14:10:07,wednesday,october,midday
1,N/A,t3_rcb83,I spent the last money I had on gas today. Im ...,"[Request] California, No cash and I could use ...",501.111100,0.000000,0,0,15,0,...,34,116,fohacidal,1332652424,1332648824,2012-03-24 22:13:44,2012-03-24 21:13:44,saturday,march,late_night
2,N/A,t3_lpu5j,My girlfriend decided it would be a good idea ...,"[Request] Hungry couple in Dundee, Scotland wo...",0.000000,0.000000,0,0,0,0,...,0,0,jacquibatman7,1319650094,1319646494,2011-10-26 10:28:14,2011-10-26 09:28:14,wednesday,october,morning
3,N/A,t3_mxvj3,"It's cold, I'n hungry, and to be completely ho...","[Request] In Canada (Ontario), just got home f...",6.518438,0.000000,36,0,1,0,...,54,76,4on_the_floor,1322855434,1322855434,2011-12-02 11:50:34,2011-12-02 11:50:34,friday,december,midday
4,N/A,t3_1i6486,hey guys:\n I love this sub. I think it's grea...,[Request] Old friend coming to visit. Would LO...,162.063252,101.606505,140,2,14,0,...,1121,1733,Futuredogwalker,1373657691,1373654091,2013-07-12 12:34:51,2013-07-12 11:34:51,friday,july,midday
5,N/A,t3_14gmeb,Feeling under the weather so I called out off ...,[REQUEST] I'll give a two week xbox live code ...,582.776586,340.819329,63,1,24,0,...,234,814,jamespweb,1354911700,1354911700,2012-12-07 12:21:40,2012-12-07 12:21:40,friday,december,midday
6,N/A,t3_wcw5m,We're in Tampa Florida...moving to Ybor on Fri...,[Request] Help me give back to my roomies on F...,0.000000,0.000000,0,0,0,0,...,0,0,usftampa,1341969695,1341966095,2012-07-10 18:21:35,2012-07-10 17:21:35,tuesday,july,evening
7,N/A,t3_of16d,"(Request) I have given a few things on reddit,...","random acts of pizza, i have a request, if not...",153.249560,0.000000,0,0,34,0,...,1153,7055,thebraus,1326429793,1326429793,2012-01-12 20:43:13,2012-01-12 20:43:13,thursday,january,evening
8,N/A,t3_1ioo1k,"Wasnt really sure what to put as the title, un...","[Request] Queensland Australia, Recently moved...",366.525625,0.000000,57,0,5,0,...,189,331,The__Doctor__,1374309042,1374305442,2013-07-20 01:30:42,2013-07-20 00:30:42,saturday,july,morning
9,N/A,t3_k0l9j,"Austin, Texas\n\nMy two roommates and I are hu...",[REQUEST]We're in need of some om noms...,348.920428,0.000000,2,0,0,0,...,6,6,biffle,1314827982,1314824382,2011-08-31 14:59:42,2011-08-31 13:59:42,wednesday,august,midday


In [114]:
print('Train dataset columns')
df1.columns
print('Test dataset columns')
tstdf1.columns

Train dataset columns
Test dataset columns


Index(['giver_username_if_known', 'request_id', 'request_text_edit_aware',
       'request_title', 'requester_account_age_in_days_at_request',
       'requester_days_since_first_post_on_raop_at_request',
       'requester_number_of_comments_at_request',
       'requester_number_of_comments_in_raop_at_request',
       'requester_number_of_posts_at_request',
       'requester_number_of_posts_on_raop_at_request',
       'requester_number_of_subreddits_at_request',
       'requester_subreddits_at_request',
       'requester_upvotes_minus_downvotes_at_request',
       'requester_upvotes_plus_downvotes_at_request', 'requester_username',
       'unix_timestamp_of_request', 'unix_timestamp_of_request_utc',
       'human_readable_local_time', 'human_readable_UTC_time', 'weekday',
       'month', 'time_of_day'],
      dtype='object')

In [18]:
#items to binarize:
# all categorical
# requester_subreddits_at_request
# weekday
# month
# time_of_day

#add-ons:
#make bins for numeric cols to coerece into categorical
# requester_account_age_in_days_at_request
# requester_days_since_first_post_on_raop_at_request
# requester_number_of_comments_at_request
# requester_number_of_comments_in_raop_at_request
# requester_number_of_posts_at_request
# requester_number_of_posts_on_raop_at_request
# requester_number_of_subreddits_at_request
# requester_upvotes_minus_downvotes_at_request
# requester_upvotes_plus_downvotes_at_request



In [115]:
df1.head()

,giver_username_if_known,request_id,request_text_edit_aware,request_title,requester_account_age_in_days_at_request,requester_days_since_first_post_on_raop_at_request,requester_number_of_comments_at_request,requester_number_of_comments_in_raop_at_request,requester_number_of_posts_at_request,requester_number_of_posts_on_raop_at_request,...,requester_upvotes_minus_downvotes_at_request,requester_upvotes_plus_downvotes_at_request,requester_username,unix_timestamp_of_request,unix_timestamp_of_request_utc,human_readable_local_time,human_readable_UTC_time,weekday,month,time_of_day
0,N/A,t3_l25d7,Hi I am in need of food for my 4 children we a...,Request Colorado Springs Help Us Please,0.000000,0.000000,0,0,0,0,...,0,0,nickylvst,1317852607,1317849007,2011-10-05 15:10:07,2011-10-05 14:10:07,wednesday,october,midday
1,N/A,t3_rcb83,I spent the last money I had on gas today. Im ...,"[Request] California, No cash and I could use ...",501.111100,0.000000,0,0,15,0,...,34,116,fohacidal,1332652424,1332648824,2012-03-24 22:13:44,2012-03-24 21:13:44,saturday,march,late_night
2,N/A,t3_lpu5j,My girlfriend decided it would be a good idea ...,"[Request] Hungry couple in Dundee, Scotland wo...",0.000000,0.000000,0,0,0,0,...,0,0,jacquibatman7,1319650094,1319646494,2011-10-26 10:28:14,2011-10-26 09:28:14,wednesday,october,morning
3,N/A,t3_mxvj3,"It's cold, I'n hungry, and to be completely ho...","[Request] In Canada (Ontario), just got home f...",6.518438,0.000000,36,0,1,0,...,54,76,4on_the_floor,1322855434,1322855434,2011-12-02 11:50:34,2011-12-02 11:50:34,friday,december,midday
4,N/A,t3_1i6486,hey guys:\n I love this sub. I think it's grea...,[Request] Old friend coming to visit. Would LO...,162.063252,101.606505,140,2,14,0,...,1121,1733,Futuredogwalker,1373657691,1373654091,2013-07-12 12:34:51,2013-07-12 11:34:51,friday,july,midday


In [116]:
tstdf1.head()

,giver_username_if_known,request_id,request_text_edit_aware,request_title,requester_account_age_in_days_at_request,requester_days_since_first_post_on_raop_at_request,requester_number_of_comments_at_request,requester_number_of_comments_in_raop_at_request,requester_number_of_posts_at_request,requester_number_of_posts_on_raop_at_request,...,requester_upvotes_minus_downvotes_at_request,requester_upvotes_plus_downvotes_at_request,requester_username,unix_timestamp_of_request,unix_timestamp_of_request_utc,human_readable_local_time,human_readable_UTC_time,weekday,month,time_of_day
0,N/A,t3_i8iy4,Hey all! It's about 95 degrees here and our ki...,[request] pregger gf 95 degree house and no fo...,42.083866,0.0,57,0,10,0,...,364,840,j_like,1308963419,1308959819,2011-06-24 17:56:59,2011-06-24 16:56:59,friday,june,evening
1,N/A,t3_1mfqi0,I didn't know a place like this exists! \n\nI ...,"[Request] Lost my job day after labour day, st...",223.784537,0.0,145,0,36,0,...,516,1448,0110110101101100,1379263523,1379259923,2013-09-15 09:45:23,2013-09-15 08:45:23,sunday,september,morning
2,N/A,t3_lclka,Hi Reddit. Im a single dad having a really rou...,(Request) pizza for my kids please?,0.000000,0.0,0,0,0,0,...,0,0,singledad22601,1318636421,1318632821,2011-10-14 16:53:41,2011-10-14 15:53:41,friday,october,midday
3,N/A,t3_1jdgdj,Hi I just moved to Waltham MA from my home sta...,[Request] Just moved to a new state(Waltham MA...,481.311273,0.0,277,0,17,0,...,1058,2062,Neuronut,1375220282,1375216682,2013-07-30 14:38:02,2013-07-30 13:38:02,tuesday,july,midday
4,N/A,t3_t2qt4,We're just sitting here near indianapolis on o...,"[Request] Two girls in between paychecks, we'v...",0.000000,0.0,0,0,0,0,...,0,0,so_damn_hungry,1335934358,1335930758,2012-05-01 21:52:38,2012-05-01 20:52:38,tuesday,may,evening


In [117]:
b = pd.get_dummies(df1['weekday'])
tstb = pd.get_dummies(tstdf1['weekday'])

In [118]:
c = pd.get_dummies(df1['time_of_day'])
tstc = pd.get_dummies(tstdf1['time_of_day'])

In [119]:
d= pd.get_dummies(df1['month'])
tstd= pd.get_dummies(tstdf1['month'])

In [120]:
df2 = pd.concat([df1,b,c,d], axis=1)
tstdf2 = pd.concat([tstdf1,tstb,tstc,tstd], axis=1)

In [121]:
df2

,giver_username_if_known,request_id,request_text_edit_aware,request_title,requester_account_age_in_days_at_request,requester_days_since_first_post_on_raop_at_request,requester_number_of_comments_at_request,requester_number_of_comments_in_raop_at_request,requester_number_of_posts_at_request,requester_number_of_posts_on_raop_at_request,...,december,february,january,july,june,march,may,november,october,september
0,N/A,t3_l25d7,Hi I am in need of food for my 4 children we a...,Request Colorado Springs Help Us Please,0.000000,0.000000,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,N/A,t3_rcb83,I spent the last money I had on gas today. Im ...,"[Request] California, No cash and I could use ...",501.111100,0.000000,0,0,15,0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2,N/A,t3_lpu5j,My girlfriend decided it would be a good idea ...,"[Request] Hungry couple in Dundee, Scotland wo...",0.000000,0.000000,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,N/A,t3_mxvj3,"It's cold, I'n hungry, and to be completely ho...","[Request] In Canada (Ontario), just got home f...",6.518438,0.000000,36,0,1,0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,N/A,t3_1i6486,hey guys:\n I love this sub. I think it's grea...,[Request] Old friend coming to visit. Would LO...,162.063252,101.606505,140,2,14,0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
5,N/A,t3_14gmeb,Feeling under the weather so I called out off ...,[REQUEST] I'll give a two week xbox live code ...,582.776586,340.819329,63,1,24,0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,N/A,t3_wcw5m,We're in Tampa Florida...moving to Ybor on Fri...,[Request] Help me give back to my roomies on F...,0.000000,0.000000,0,0,0,0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
7,N/A,t3_of16d,"(Request) I have given a few things on reddit,...","random acts of pizza, i have a request, if not...",153.249560,0.000000,0,0,34,0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,N/A,t3_1ioo1k,"Wasnt really sure what to put as the title, un...","[Request] Queensland Australia, Recently moved...",366.525625,0.000000,57,0,5,0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
9,N/A,t3_k0l9j,"Austin, Texas\n\nMy two roommates and I are hu...",[REQUEST]We're in need of some om noms...,348.920428,0.000000,2,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [122]:
del df2['month']
del tstdf2['month']

In [123]:
del df2['weekday']
del tstdf2['weekday']

In [124]:
del df2['time_of_day']
del tstdf2['time_of_day']

In [125]:
subr = []
for x in df1['requester_subreddits_at_request']:
    for i, t in enumerate(x):
            subr.append(t)
subr = set(subr)

tstsubr = []
for x in tstdf1['requester_subreddits_at_request']:
    for i, t in enumerate(x):
            tstsubr.append(t)
tstsubr = set(tstsubr)

In [126]:
print(len(subr),len(tstsubr) )

7484 4431


In [127]:
df3 = pd.get_dummies(df2['requester_subreddits_at_request'].apply(pd.Series).stack()).sum(level=0)
tstdf3 = pd.get_dummies(tstdf2['requester_subreddits_at_request'].apply(pd.Series).stack()).sum(level=0)

In [128]:
print(df3.shape, tstdf3.shape)

(3311, 7484) (1334, 4431)


In [129]:
print(df2.shape, tstdf2.shape)

(4040, 42) (1631, 42)


In [130]:
df3.head()

,0x10c,0x10cships,1000words,100movies365days,100pushups,100sets,1200isplenty,14thCenturyEngland,15thcenturyproblems,1911,...,yugijerk,yugioh,zeddit,zelda,zen,zoidberg,zombiefortress,zombies,zombiespartyup,zoology
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [131]:
tstdf3.head()

,1000words,1920s,195,1985sweet1985,2007scape,2012Elections,24hoursupport,250r,29er,2GOINTR,...,zanarchy,zanarchysubmissions,zelda,zeldaconspiracies,zeldents,zen,zoidberg,zombiemanic,zombies,zombiespartyup
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [132]:
df4 = pd.get_dummies(df2['requester_subreddits_at_request'].apply(pd.Series), prefix='', prefix_sep='').sum(level=0, axis=1)
tstdf4 = pd.get_dummies(tstdf2['requester_subreddits_at_request'].apply(pd.Series), prefix='', prefix_sep='').sum(level=0, axis=1)

In [133]:
print(df4.shape, tstdf4.shape)

(4040, 7484) (1631, 4431)


In [134]:
df5 = pd.concat([df2,df4], axis = 1)
tstdf5 = pd.concat([tstdf2,tstdf4], axis = 1)

In [135]:
df5.head()

,giver_username_if_known,request_id,request_text_edit_aware,request_title,requester_account_age_in_days_at_request,requester_days_since_first_post_on_raop_at_request,requester_number_of_comments_at_request,requester_number_of_comments_in_raop_at_request,requester_number_of_posts_at_request,requester_number_of_posts_on_raop_at_request,...,yugijerk,yugioh,zeddit,zelda,zen,zoidberg,zombiefortress,zombies,zombiespartyup,zoology
0,N/A,t3_l25d7,Hi I am in need of food for my 4 children we a...,Request Colorado Springs Help Us Please,0.000000,0.000000,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,N/A,t3_rcb83,I spent the last money I had on gas today. Im ...,"[Request] California, No cash and I could use ...",501.111100,0.000000,0,0,15,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,N/A,t3_lpu5j,My girlfriend decided it would be a good idea ...,"[Request] Hungry couple in Dundee, Scotland wo...",0.000000,0.000000,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,N/A,t3_mxvj3,"It's cold, I'n hungry, and to be completely ho...","[Request] In Canada (Ontario), just got home f...",6.518438,0.000000,36,0,1,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,N/A,t3_1i6486,hey guys:\n I love this sub. I think it's grea...,[Request] Old friend coming to visit. Would LO...,162.063252,101.606505,140,2,14,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [136]:
df2 = df2[[ 'request_id','friday',
       'monday', 'saturday', 'sunday', 'thursday', 'tuesday', 'wednesday',
       'evening', 'late_night', 'midday', 'morning', 'april', 'august',
       'december', 'february', 'january', 'july', 'june', 'march', 'may',
       'november', 'october', 'september']]
tstdf2 = tstdf2[[ 'request_id','friday',
       'monday', 'saturday', 'sunday', 'thursday', 'tuesday', 'wednesday',
       'evening', 'late_night', 'midday', 'morning', 'april', 'august',
       'december', 'february', 'january', 'july', 'june', 'march', 'may',
       'november', 'october', 'september']]

In [137]:
df2.columns

Index(['request_id', 'friday', 'monday', 'saturday', 'sunday', 'thursday',
       'tuesday', 'wednesday', 'evening', 'late_night', 'midday', 'morning',
       'april', 'august', 'december', 'february', 'january', 'july', 'june',
       'march', 'may', 'november', 'october', 'september'],
      dtype='object')

In [138]:
df5 = pd.concat([a['requester_received_pizza'],df2,df4], axis = 1)

In [139]:
df5.head()

,requester_received_pizza,request_id,friday,monday,saturday,sunday,thursday,tuesday,wednesday,evening,...,yugijerk,yugioh,zeddit,zelda,zen,zoidberg,zombiefortress,zombies,zombiespartyup,zoology
0,False,t3_l25d7,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,False,t3_rcb83,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,False,t3_lpu5j,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,False,t3_mxvj3,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,False,t3_1i6486,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [140]:
df5.to_json('binarized_categorical_data.json')
tstdf5.to_json('binarized_categorical_test_data.json')

In [141]:
df6 = pd.read_json('binarized_categorical_data.json')
tstdf6 = pd.read_json('binarized_categorical_test_data.json')

In [142]:
df6[['requester_received_pizza','monday','april','december','midday']].head()

,requester_received_pizza,monday,april,december,midday
0,False,0,0,0,1
1,False,0,0,0,0
10,True,0,0,0,0
100,False,0,1,0,1
1000,False,0,0,0,0


In [143]:
del df6['request_id']
print(df6.shape)

(4040, 7508)


In [144]:
del tstdf6['request_id']
print(tstdf6.shape)

(1631, 4472)


Divide the Binarized Categorical Data - Train into Train and Dev datasets - We will use the Dev dataset for testing the accuracy of the prediction

In [145]:
df6_labels = df6['requester_received_pizza']
del df6['requester_received_pizza']
df6_data = df6

bincat_train_data = df6_data[0:2019]
bincat_train_labels = df6_labels[0:2019]
bincat_test_data = df6_data[2020:]
bincat_test_labels = df6_labels[2020:]

print(bincat_train_data.shape, bincat_test_data.shape, bincat_train_labels.shape, bincat_test_labels.shape)

(2019, 7507) (2020, 7507) (2019,) (2020,)


We run a series of experiments to find the model that gives the best accuracy with no more than 50 parameters.

In [146]:
def calc_param(n_features, n_components, covtype, classes):
## Function to compute the total parameters for model
   if covtype == 'full': #Covariance Parameters (n_components, n_features, n_features) Symmetric matrix
        totparam = (n_features*n_components + (n_features*(n_features+1)/2)*n_components)*classes
   elif covtype == 'diag': #Covariance Parameters (n_components, n_features)    
        totparam = (n_features*n_components + n_features*n_components)*classes
   elif covtype == 'spherical': #Covariance Parameters = (n_components,)
        totparam = (n_features*n_components + 1)*classes
   elif covtype == 'tied': #Covariance Parameters (n_features, n_features)  Symmetric matrix
        totparam = (n_features*n_components + (n_features*(n_features+1)/2))*classes
   else:
      print('Invalid Covariance Type %s'%(covtype))
    
   return totparam

def P6():
### STUDENT START ###
    cov_matrix = ['spherical', 'diag', 'tied', 'full'] #Initialize list of covariance types 
    modelnum = 0 #Initialize model number
#Loop through to run a series of experiments to find the model that gives the best accuracy with no more than 50 parameters
    for comp in range(1,8): #Vary the Principal components between 1 and 8
        #Project the train data onto the reduced Principal component subspace
        sklearn_pca = PCA(n_components=comp)
        sklearn_transf = sklearn_pca.fit_transform(bincat_train_data)
        #Divide the projected data into positive and negative class datasets
        positive_train_data = sklearn_transf[bincat_train_labels == True]
        negative_train_data = sklearn_transf[bincat_train_labels == False]
        
        for GMMcomp in range(1,8): #Vary the Gaussian Mixture model components between 1 and 8
          for covtype in cov_matrix: #Loop thru the 4 covariance types for the PCA and GMM components
            totparam = calc_param(comp, GMMcomp, covtype, 2) #Calculate the total parameters for model
            modelnum += 1 #Increment the model number
            if totparam <= 50: #Validate that the total parameters for model is <= 50
            #Train the GMM positive and negative models on the positive and negative datasets respectively
                   gm_mod_pos = GMM(n_components = GMMcomp, covariance_type=covtype)
                   gm_mod_pos.fit(positive_train_data)
                   gm_mod_neg = GMM(n_components = GMMcomp, covariance_type=covtype)
                   gm_mod_neg.fit(negative_train_data)
            #Score the trained model on the test dataset
                   sklearn_test_transf = sklearn_pca.transform(bincat_test_data)
                   pscore = gm_mod_pos.score(sklearn_test_transf)
                   nscore = gm_mod_neg.score(sklearn_test_transf)
           
            #Loop to determine the predicted labels of the test data based on the trained model
                   preds = []
                   for pos, neg in zip(pscore, nscore):
                        if pos > neg: preds.append(1)
                        else: preds.append(0)
            
            #Loop to compare the predicted label with the actual label to determine the no of correct answers
                   correct = 0
                   for pred, label in zip(preds, bincat_test_labels):
                        if pred == label: correct +=1
            #Calculate and print the accuracy of the model
                   print('%d Parameters %d PCA Comp %d GMM Comp %d Cov Matrix %s Accuracy is\t %.2f%%'% (modelnum, totparam, comp, GMMcomp, covtype, (100.0*correct/len(bincat_test_labels))))
            else: #If total parameters > 50 skip
                   print('%d Parameters %d PCA Comp %d GMM Comp %d Cov Matrix %s exceeds 50 params'% (modelnum, totparam, comp, GMMcomp, covtype))

            
### STUDENT END ###

P6()

/Users/jayashreeraman/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:26: FutureWarning: in the future, boolean array-likes will be handled as a boolean array index
/Users/jayashreeraman/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:27: FutureWarning: in the future, boolean array-likes will be handled as a boolean array index


1 Parameters 4 PCA Comp 1 GMM Comp 1 Cov Matrix spherical Accuracy is	 53.27%
2 Parameters 4 PCA Comp 1 GMM Comp 1 Cov Matrix diag Accuracy is	 53.27%
3 Parameters 4 PCA Comp 1 GMM Comp 1 Cov Matrix tied Accuracy is	 53.27%
4 Parameters 4 PCA Comp 1 GMM Comp 1 Cov Matrix full Accuracy is	 53.27%
5 Parameters 6 PCA Comp 1 GMM Comp 2 Cov Matrix spherical Accuracy is	 53.27%
6 Parameters 8 PCA Comp 1 GMM Comp 2 Cov Matrix diag Accuracy is	 53.27%
7 Parameters 6 PCA Comp 1 GMM Comp 2 Cov Matrix tied Accuracy is	 53.27%
8 Parameters 8 PCA Comp 1 GMM Comp 2 Cov Matrix full Accuracy is	 53.27%
9 Parameters 8 PCA Comp 1 GMM Comp 3 Cov Matrix spherical Accuracy is	 53.27%
10 Parameters 12 PCA Comp 1 GMM Comp 3 Cov Matrix diag Accuracy is	 53.27%
11 Parameters 8 PCA Comp 1 GMM Comp 3 Cov Matrix tied Accuracy is	 53.27%
12 Parameters 12 PCA Comp 1 GMM Comp 3 Cov Matrix full Accuracy is	 53.27%
13 Parameters 10 PCA Comp 1 GMM Comp 4 Cov Matrix spherical Accuracy is	 53.27%
14 Parameters 16 PCA Com

Best accuracy for this model is :Parameters 8 PCA Comp 2 GMM Comp 1 Cov Matrix diag Accuracy is	 62.82%

We try a Random Forest classifier on the binarized categorical data

In [147]:
clf2 = RandomForestClassifier(n_estimators=12)
clf2.fit(bincat_train_data, bincat_train_labels)

preds2 = clf2.predict(bincat_test_data)

print(classification_report(bincat_test_labels,preds2))

print(metrics.accuracy_score(bincat_test_labels,preds2))

             precision    recall  f1-score   support

      False       0.76      0.95      0.84      1519
       True       0.35      0.07      0.12       501

avg / total       0.66      0.74      0.67      2020

0.736138613861
